# Full audio

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 39

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


coswara_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
coughvid_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features.npy')

### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,...,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,cough_path,label,error
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,healthy,0
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,healthy,0
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm,NaN,0
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm,healthy,0
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001c85a8-cc4d-4921-9297-848be52d4715.webm,NaN,0


In [ ]:
logger.info(meta_df.shape)

2022-05-01 03:03:07,658 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - (20072, 43)


In [ ]:
mfcc_feature = np.load(coughvid_feature)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 20072, (20072, 43))

In [ ]:
DROP_COLS = [
             'uuid',
             'error',
             'datetime',
             'cough_path',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                  8326
longitude                 8326
age                       9604
gender                    8758
respiratory_condition     8758
fever_muscle_pain         8758
status                    8758
quality_1                19252
cough_type_1             19252
dyspnea_1                19252
wheezing_1               19252
stridor_1                19252
choking_1                19252
congestion_1             19252
nothing_1                19252
diagnosis_1              19252
severity_1               19252
quality_2                19252
cough_type_2             19253
dyspnea_2                19252
wheezing_2               19252
stridor_2                19252
choking_2                19252
congestion_2             19252
nothing_2                19252
diagnosis_2              19252
severity_2               19252
quality_3                19252
dyspnea_3                19252
wheezing

In [ ]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
status                    object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3 

In [ ]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
status                   0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
c

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [ ]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-05-01 03:03:22,458 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - {0.0: 8562, 1.0: 1742}


In [ ]:
meta_df.label.value_counts().sum()

10304

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-05-01 03:03:23,468 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - 1742, 8562


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

1742


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:6850], feature_1[:1394]), axis=0)
  val = np.concatenate((feature_0[6850:7706], feature_1[1394:1568]), axis=0)
  test = np.concatenate((feature_0[7706:], feature_1[1568:]), axis=0)

  y_train = pd.concat([label0[:6850], label1[:1394]], ignore_index=True)
  y_val = pd.concat([label0[6850:7706], label1[1394:1568]], ignore_index=True)
  y_test = pd.concat([label0[7706:], label1[1568:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(8244, 234, 246)


In [ ]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
X_train_1d.shape

(8244, 37, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 88s 66ms/step - loss: 0.5011 - auc: 0.8358 - val_loss: 0.5193 - val_auc: 0.8175
Epoch 2/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4644 - auc: 0.8426 - val_loss: 0.4859 - val_auc: 0.8335
Epoch 3/40
1031/1031 [==============================] - 67s 65ms/step - loss: 0.4598 - auc: 0.8410 - val_loss: 0.4566 - val_auc: 0.8498
Epoch 4/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4561 - auc: 0.8431 - val_loss: 0.4548 - val_auc: 0.8470
Epoch 5/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4567 - auc: 0.8431 - val_loss: 0.4592 - val_auc: 0.8293
Epoch 6/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4546 - auc: 0.8430 - val_loss: 0.4549 - val_auc: 0.8282
Epoch 7/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4538 - auc: 0.8437 - val_loss: 0.4575 - val_auc: 0.8305
Epoch 8/40
33/33 [==============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 65s 110ms/step - loss: 0.5498 - auc: 0.8329 - val_loss: 0.5094 - val_auc: 0.8359
Epoch 2/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4564 - auc: 0.8443 - val_loss: 0.4753 - val_auc: 0.8351
Epoch 3/40
516/516 [==============================] - 57s 111ms/step - loss: 0.4551 - auc: 0.8454 - val_loss: 0.4503 - val_auc: 0.8526
Epoch 4/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4546 - auc: 0.8489 - val_loss: 0.4534 - val_auc: 0.8468
Epoch 5/40
516/516 [==============================] - 56s 108ms/step - loss: 0.4556 - auc: 0.8448 - val_loss: 0.4528 - val_auc: 0.8475
Epoch 6/40
516/516 [==============================] - 56s 108ms/step - loss: 0.4525 - auc: 0.8484 - val_loss: 0.4709 - val_auc: 0.8353
Epoch 7/40
516/516 [==============================] - 56s 108ms/step - loss: 0.4522 - auc: 0.8513 - val_loss: 0.4533 - val_auc: 0.8473
Epoch 8/40
33/33 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 57s 741ms/step - loss: 0.7933 - auc: 0.8350 - val_loss: 0.5950 - val_auc: 0.8265
Epoch 2/40
65/65 [==============================] - 47s 723ms/step - loss: 0.4623 - auc: 0.8424 - val_loss: 0.6132 - val_auc: 0.8358
Epoch 3/40
65/65 [==============================] - 46s 712ms/step - loss: 0.4537 - auc: 0.8486 - val_loss: 0.5793 - val_auc: 0.8408
Epoch 4/40
65/65 [==============================] - 46s 716ms/step - loss: 0.4510 - auc: 0.8526 - val_loss: 0.5830 - val_auc: 0.8424
Epoch 5/40
65/65 [==============================] - 47s 720ms/step - loss: 0.4460 - auc: 0.8598 - val_loss: 0.5773 - val_auc: 0.8010
Epoch 6/40
65/65 [==============================] - 47s 721ms/step - loss: 0.4488 - auc: 0.8589 - val_loss: 0.5852 - val_auc: 0.8313
Epoch 7/40
65/65 [==============================] - 47s 723ms/step - loss: 0.4456 - auc: 0.8614 - val_loss: 0.6042 - val_auc: 0.7721
Epoch 8/40
65/65 [==============================] - 47s 721ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([8244, 2])

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 78s 65ms/step - loss: 0.5055 - auc: 0.8355 - val_loss: 0.4750 - val_auc: 0.8294
Epoch 2/40
1031/1031 [==============================] - 64s 63ms/step - loss: 0.4668 - auc: 0.8405 - val_loss: 0.5552 - val_auc: 0.7949
Epoch 3/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4613 - auc: 0.8440 - val_loss: 0.4703 - val_auc: 0.8398
Epoch 4/40
1031/1031 [==============================] - 65s 64ms/step - loss: 0.4576 - auc: 0.8452 - val_loss: 0.4515 - val_auc: 0.8513
Epoch 5/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4550 - auc: 0.8438 - val_loss: 0.4622 - val_auc: 0.8315
Epoch 6/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4557 - auc: 0.8425 - val_loss: 0.4540 - val_auc: 0.8421
Epoch 7/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4553 - auc: 0.8457 - val_loss: 0.4695 - val_auc: 0.8332
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 65s 110ms/step - loss: 0.5864 - auc: 0.8387 - val_loss: 0.4644 - val_auc: 0.8246
Epoch 2/40
516/516 [==============================] - 55s 106ms/step - loss: 0.4591 - auc: 0.8433 - val_loss: 0.4547 - val_auc: 0.8421
Epoch 3/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4541 - auc: 0.8492 - val_loss: 0.4570 - val_auc: 0.8471
Epoch 4/40
516/516 [==============================] - 55s 106ms/step - loss: 0.4534 - auc: 0.8492 - val_loss: 0.4576 - val_auc: 0.8447
Epoch 5/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4546 - auc: 0.8500 - val_loss: 0.4569 - val_auc: 0.8395
Epoch 6/40
516/516 [==============================] - 55s 106ms/step - loss: 0.4519 - auc: 0.8530 - val_loss: 0.4561 - val_auc: 0.8430
Epoch 7/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4495 - auc: 0.8550 - val_loss: 0.4681 - val_auc: 0.8387
Epoch 8/40
33/33 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 57s 746ms/step - loss: 0.8307 - auc: 0.8080 - val_loss: 0.6101 - val_auc: 0.8263
Epoch 2/40
65/65 [==============================] - 47s 721ms/step - loss: 0.4555 - auc: 0.8479 - val_loss: 0.6428 - val_auc: 0.8335
Epoch 3/40
65/65 [==============================] - 47s 720ms/step - loss: 0.4500 - auc: 0.8550 - val_loss: 0.6617 - val_auc: 0.8361
Epoch 4/40
65/65 [==============================] - 46s 712ms/step - loss: 0.4495 - auc: 0.8574 - val_loss: 0.6066 - val_auc: 0.8334
Epoch 5/40
65/65 [==============================] - 47s 721ms/step - loss: 0.4429 - auc: 0.8622 - val_loss: 0.5804 - val_auc: 0.8045
Epoch 6/40
65/65 [==============================] - 47s 725ms/step - loss: 0.4388 - auc: 0.8682 - val_loss: 0.5789 - val_auc: 0.8219
Epoch 7/40
65/65 [==============================] - 47s 726ms/step - loss: 0.4350 - auc: 0.8708 - val_loss: 0.5695 - val_auc: 0.8336
Epoch 8/40
33/33 [==============================] - 1s 28ms/step - lo

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(13700, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(13700, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([13700, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1713/1713 [==============================] - 138s 69ms/step - loss: 0.5943 - auc: 0.7585 - val_loss: 1.0266 - val_auc: 0.4900
Epoch 2/40
1713/1713 [==============================] - 117s 68ms/step - loss: 0.5076 - auc: 0.8266 - val_loss: 0.5299 - val_auc: 0.8295
Epoch 3/40
1713/1713 [==============================] - 116s 68ms/step - loss: 0.4813 - auc: 0.8469 - val_loss: 0.4827 - val_auc: 0.8369
Epoch 4/40
1713/1713 [==============================] - 116s 68ms/step - loss: 0.4556 - auc: 0.8630 - val_loss: 0.4884 - val_auc: 0.8481
Epoch 5/40
1713/1713 [==============================] - 115s 67ms/step - loss: 0.4275 - auc: 0.8807 - val_loss: 0.4858 - val_auc: 0.8371
Epoch 6/40
1713/1713 [==============================] - 116s 68ms/step - loss: 0.4132 - auc: 0.8887 - val_loss: 0.5584 - val_auc: 0.8389
Epoch 7/40
1713/1713 [==============================] - 116s 67ms/step - loss: 0.3990 - auc: 0.8971 - val_loss: 1.0070 - val_auc: 0.5316
Epoch 8/40
1713/1713 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
857/857 [==============================] - 116s 126ms/step - loss: 0.6544 - auc: 0.7326 - val_loss: 0.5378 - val_auc: 0.8093
Epoch 2/40
857/857 [==============================] - 105s 123ms/step - loss: 0.4952 - auc: 0.8374 - val_loss: 0.6080 - val_auc: 0.7787
Epoch 3/40
857/857 [==============================] - 105s 123ms/step - loss: 0.4596 - auc: 0.8616 - val_loss: 0.4659 - val_auc: 0.8437
Epoch 4/40
857/857 [==============================] - 105s 123ms/step - loss: 0.4332 - auc: 0.8784 - val_loss: 1.2618 - val_auc: 0.4472
Epoch 5/40
857/857 [==============================] - 105s 123ms/step - loss: 0.4195 - auc: 0.8861 - val_loss: 0.5246 - val_auc: 0.8173
Epoch 6/40
857/857 [==============================] - 105s 123ms/step - loss: 0.3992 - auc: 0.8980 - val_loss: 0.6756 - val_auc: 0.7213
Epoch 7/40
857/857 [==============================] - 105s 123ms/step - loss: 0.3748 - auc: 0.9099 - val_loss: 0.5773 - val_auc: 0.8347
Epoch 8/40
33/33 [==============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
108/108 [==============================] - 109s 914ms/step - loss: 0.8504 - auc: 0.6885 - val_loss: 1.8048 - val_auc: 0.1737
Epoch 2/40
108/108 [==============================] - 98s 903ms/step - loss: 0.4945 - auc: 0.8378 - val_loss: 0.9804 - val_auc: 0.1768
Epoch 3/40
108/108 [==============================] - 98s 908ms/step - loss: 0.4612 - auc: 0.8587 - val_loss: 1.3139 - val_auc: 0.1786
Epoch 4/40
108/108 [==============================] - 98s 909ms/step - loss: 0.4452 - auc: 0.8682 - val_loss: 1.2396 - val_auc: 0.1696
Epoch 5/40
108/108 [==============================] - 98s 908ms/step - loss: 0.3973 - auc: 0.8969 - val_loss: 2.0168 - val_auc: 0.1600
Epoch 6/40
108/108 [==============================] - 98s 906ms/step - loss: 0.3917 - auc: 0.9004 - val_loss: 4.2015 - val_auc: 0.1683
Epoch 7/40
108/108 [==============================] - 97s 897ms/step - loss: 0.3743 - auc: 0.9095 - val_loss: 0.4889 - val_auc: 0.8376
Epoch 8/40
108/108 [==============================] - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 10s 51ms/step - loss: 0.5773 - auc: 0.7988


[0.5772629380226135, 0.7987561821937561]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 3s 44ms/step - loss: 0.4637 - auc: 0.8473


[0.4637441635131836, 0.8472683429718018]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 3s 43ms/step - loss: 0.5042 - auc: 0.8303


[0.5041710138320923, 0.8302884697914124]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 0.4565 - auc: 0.8406


[0.45653754472732544, 0.8405871987342834]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 0.4541 - auc: 0.8374


[0.4540819227695465, 0.8374314904212952]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 0.4930 - auc: 0.8399


[0.49299296736717224, 0.8398727774620056]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 44ms/step - loss: 0.4536 - auc: 0.8502


[0.4535810649394989, 0.850238561630249]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 4s 45ms/step - loss: 0.4522 - auc: 0.8506


[0.4522389769554138, 0.8505542874336243]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 0.4502 - auc: 0.8568


[0.45015543699264526, 0.8568271398544312]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 46ms/step - loss: 0.4515 - auc: 0.8510


[0.4515165388584137, 0.8509839773178101]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 46ms/step - loss: 0.7768 - auc: 0.7922


[0.7768452763557434, 0.792221188545227]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 0.6124 - auc: 0.8324


[0.6124073266983032, 0.8324451446533203]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 45ms/step - loss: 1.3917 - auc: 0.5782


[1.391661286354065, 0.578154444694519]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 44ms/step - loss: 0.5632 - auc: 0.8224


[0.5632295608520508, 0.8223904371261597]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 46ms/step - loss: 0.6705 - auc: 0.8257


[0.6705477237701416, 0.8257220387458801]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 39

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

coughvid_feature = [
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy'),
]


### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,quality_1,...,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,label,sr_cough,chunk_path,num_chunk
0,00039425-7f3a-42aa-ac13-834aaa2b6b92-0,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav,3
1,00039425-7f3a-42aa-ac13-834aaa2b6b92-1,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-1.wav,3
2,00039425-7f3a-42aa-ac13-834aaa2b6b92-2,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-2.wav,3
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0.wav,3
4,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1.wav,3


In [5]:
logger.info(meta_df.shape)

2022-05-03 06:44:20,327 - COUGHVID - HYBRID MODEL CHUNKS - INFO - (84981, 43)


In [6]:
DROP_COLS = [
             'uuid',
             'num_chunk',
             'datetime',
             'chunk_path',
             'sr_cough',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                 33197
longitude                33197
age                      35707
gender                   31986
respiratory_condition    31986
fever_muscle_pain        31986
quality_1                80743
cough_type_1             80743
dyspnea_1                80743
wheezing_1               80743
stridor_1                80743
choking_1                80743
congestion_1             80743
nothing_1                80743
diagnosis_1              80743
severity_1               80743
quality_2                80643
cough_type_2             80651
dyspnea_2                80643
wheezing_2               80643
stridor_2                80643
choking_2                80643
congestion_2             80643
nothing_2                80643
diagnosis_2              80643
severity_2               80643
quality_3                80594
dyspnea_3                80594
wheezing_3               80594
stridor_

In [7]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3                 object
dyspnea_3 

In [8]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
cough_type_3             0
d

In [9]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [10]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-05-03 06:44:20,871 - COUGHVID - HYBRID MODEL CHUNKS - INFO - {0.0: 39866, 1.0: 13129}


In [11]:
meta_df.label.value_counts().sum()

52995

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [13]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [14]:
mfcc_feature_1 = np.load(coughvid_feature[0])
mfcc_feature_2 = np.load(coughvid_feature[1])
mfcc_feature_3 = np.load(coughvid_feature[2])
mfcc_feature_4 = np.load(coughvid_feature[3])
mfcc_feature_5 = np.load(coughvid_feature[4])

##### 2d

In [15]:
null_label = meta_df[meta_df['label'].isna() == False].index.tolist() # index of records not n/a labels
meta_df.dropna(inplace=True)
meta_df.reset_index(drop=True, inplace=True)

In [16]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [17]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-05-03 06:45:37,239 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 13129, 39866


In [18]:
# remove features of null labels
mfcc_feature_1 = mfcc_feature_1[[i for i in null_label if i < 20000]]
mfcc_feature_2 = mfcc_feature_2[[i - 20000 for i in null_label if i >= 20000 and i < 40000]]
mfcc_feature_3 = mfcc_feature_3[[i - 40000 for i in null_label if i >= 40000 and i < 60000]]
mfcc_feature_4 = mfcc_feature_4[[i - 60000 for i in null_label if i >= 60000 and i < 75000]]
mfcc_feature_5 = mfcc_feature_5[[i - 75000 for i in null_label if i >= 75000]]
mfcc_features = np.concatenate((mfcc_feature_1, mfcc_feature_2, mfcc_feature_3, mfcc_feature_4, mfcc_feature_5), axis=0)

In [19]:
# release RAM
del mfcc_feature_1
del mfcc_feature_2
del mfcc_feature_3
del mfcc_feature_4
del mfcc_feature_5

In [20]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_0)}, {len(index_1)}')

2022-05-03 06:45:55,241 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 39866, 13129


In [21]:
mfcc_features.shape

(52995, 234, 236)

In [22]:
# get features for each class
feature_0 = mfcc_features[index_0]
feature_1 = mfcc_features[index_1]
del mfcc_features

In [23]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:31893], feature_1[:10517]), axis=0)
  val = np.concatenate((feature_0[31893:35879], feature_1[10517:11824]), axis=0)
  test = np.concatenate((feature_0[35879:], feature_1[11824:]), axis=0)

  y_train = pd.concat([label_0[:31893], label_1[:10517]], ignore_index=True)
  y_val = pd.concat([label_0[31893:35879], label_1[10517:11824]], ignore_index=True)
  y_test = pd.concat([label_0[35879:], label_1[11824:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [24]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(42410, 234, 236)


In [25]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [26]:
del feature_0
del feature_1

##### 1d

In [27]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [28]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [29]:
X_train_2d = X_train_2d[20000:]
y_train_2d = y_train_2d[20000:]
X_train_2d.shape, y_train_2d.shape

((22410, 234, 236), (22410,))

In [30]:
X_train_1d = X_train_1d[20000:]
y_train_1d = y_train_1d[20000:]
X_train_1d.shape, y_train_1d.shape

((22410, 36), (22410,))

In [31]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [33]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(32410, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(32410, 36, 1)

In [35]:
X_train_1d.shape

(32410, 36, 1)

In [36]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [40]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [41]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
4052/4052 [==============================] - 262s 61ms/step - loss: 0.6507 - auc: 0.6729 - val_loss: 0.5748 - val_auc: 0.7642
Epoch 2/40
4052/4052 [==============================] - 243s 60ms/step - loss: 0.6336 - auc: 0.6757 - val_loss: 0.5647 - val_auc: 0.7625
Epoch 3/40
4052/4052 [==============================] - 243s 60ms/step - loss: 0.6326 - auc: 0.6768 - val_loss: 0.5746 - val_auc: 0.7631
Epoch 4/40
4052/4052 [==============================] - 243s 60ms/step - loss: 0.6323 - auc: 0.6760 - val_loss: 0.5787 - val_auc: 0.7442
Epoch 5/40
4052/4052 [==============================] - 243s 60ms/step - loss: 0.6324 - auc: 0.6739 - val_loss: 0.5742 - val_auc: 0.7663
Epoch 6/40
4052/4052 [==============================] - 242s 60ms/step - loss: 0.6322 - auc: 0.6729 - val_loss: 0.5707 - val_auc: 0.7671
Epoch 7/40
4052/4052 [==============================] - 243s 60ms/step - loss: 0.6316 - auc: 0.6776 - val_loss: 0.5707 - val_auc: 0.7665
Epoch 8/40
4052/4052 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
2026/2026 [==============================] - 214s 102ms/step - loss: 0.6515 - auc: 0.6909 - val_loss: 0.5759 - val_auc: 0.7778
Epoch 2/40
2026/2026 [==============================] - 205s 101ms/step - loss: 0.6297 - auc: 0.6901 - val_loss: 0.5689 - val_auc: 0.7809
Epoch 3/40
2026/2026 [==============================] - 204s 101ms/step - loss: 0.6295 - auc: 0.6915 - val_loss: 0.5637 - val_auc: 0.7801
Epoch 4/40
2026/2026 [==============================] - 205s 101ms/step - loss: 0.6290 - auc: 0.6934 - val_loss: 0.5647 - val_auc: 0.7815
Epoch 5/40
2026/2026 [==============================] - 204s 101ms/step - loss: 0.6266 - auc: 0.6995 - val_loss: 0.5608 - val_auc: 0.7785
Epoch 6/40
2026/2026 [==============================] - 205s 101ms/step - loss: 0.6258 - auc: 0.7014 - val_loss: 0.5660 - val_auc: 0.7843
Epoch 7/40
2026/2026 [==============================] - 204s 101ms/step - loss: 0.6241 - auc: 0.7051 - val_loss: 0.5638 - val_auc: 0.7792
Epoch 8/40
2026/2026 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
254/254 [==============================] - 177s 670ms/step - loss: 0.7330 - auc: 0.6825 - val_loss: 0.6510 - val_auc: 0.7528
Epoch 2/40
254/254 [==============================] - 171s 671ms/step - loss: 0.6316 - auc: 0.6846 - val_loss: 0.6422 - val_auc: 0.7402
Epoch 3/40
254/254 [==============================] - 167s 659ms/step - loss: 0.6262 - auc: 0.6999 - val_loss: 0.6002 - val_auc: 0.7711
Epoch 4/40
254/254 [==============================] - 169s 664ms/step - loss: 0.6210 - auc: 0.7117 - val_loss: 0.5836 - val_auc: 0.7746
Epoch 5/40
254/254 [==============================] - 169s 664ms/step - loss: 0.6164 - auc: 0.7193 - val_loss: 0.5656 - val_auc: 0.7735
Epoch 6/40
254/254 [==============================] - 167s 659ms/step - loss: 0.6125 - auc: 0.7246 - val_loss: 0.5706 - val_auc: 0.7768
Epoch 7/40
254/254 [==============================] - 168s 660ms/step - loss: 0.6047 - auc: 0.7345 - val_loss: 0.6029 - val_auc: 0.7674
Epoch 8/40
254/254 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [32]:
X_train_2d.shape

(22410, 234, 236)

In [33]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(22410, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(22410, 36, 1)

In [35]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [36]:
y_train_2d.shape

TensorShape([22410, 2])

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [41]:
fp = f'./coughvid_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2802/2802 [==============================] - 190s 63ms/step - loss: 0.7115 - auc: 0.5170 - val_loss: 0.6708 - val_auc: 0.6616
Epoch 2/40
2802/2802 [==============================] - 171s 61ms/step - loss: 0.6948 - auc: 0.5213 - val_loss: 0.6708 - val_auc: 0.7513
Epoch 3/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6949 - auc: 0.5194 - val_loss: 0.6774 - val_auc: 0.5748
Epoch 4/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6950 - auc: 0.5191 - val_loss: 0.6637 - val_auc: 0.7330
Epoch 5/40
2802/2802 [==============================] - 169s 60ms/step - loss: 0.6949 - auc: 0.5194 - val_loss: 0.6715 - val_auc: 0.7414
Epoch 6/40
2802/2802 [==============================] - 168s 60ms/step - loss: 0.6942 - auc: 0.5224 - val_loss: 0.6908 - val_auc: 0.5820
Epoch 7/40
166/166 [==============================] - 5s 27ms/step - loss: 0.6536 - auc: 0.7442


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
1401/1401 [==============================] - 152s 103ms/step - loss: 0.7357 - auc: 0.5634 - val_loss: 0.6502 - val_auc: 0.7700
Epoch 2/40
1401/1401 [==============================] - 142s 101ms/step - loss: 0.6939 - auc: 0.5248 - val_loss: 0.6726 - val_auc: 0.7207
Epoch 3/40
1401/1401 [==============================] - 143s 102ms/step - loss: 0.6938 - auc: 0.5295 - val_loss: 0.6675 - val_auc: 0.6547
Epoch 4/40
1401/1401 [==============================] - 142s 102ms/step - loss: 0.6935 - auc: 0.5255 - val_loss: 0.6458 - val_auc: 0.7484
Epoch 5/40
1401/1401 [==============================] - 143s 102ms/step - loss: 0.6935 - auc: 0.5270 - val_loss: 0.6608 - val_auc: 0.7574
Epoch 6/40
166/166 [==============================] - 5s 27ms/step - loss: 0.6596 - auc: 0.7514


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
176/176 [==============================] - 127s 682ms/step - loss: 0.8407 - auc: 0.5593 - val_loss: 0.6594 - val_auc: 0.7586
Epoch 2/40
176/176 [==============================] - 118s 669ms/step - loss: 0.6935 - auc: 0.5250 - val_loss: 0.6449 - val_auc: 0.7654
Epoch 3/40
176/176 [==============================] - 118s 671ms/step - loss: 0.6932 - auc: 0.5261 - val_loss: 0.6577 - val_auc: 0.7419
Epoch 4/40
176/176 [==============================] - 118s 672ms/step - loss: 0.6927 - auc: 0.5272 - val_loss: 0.6680 - val_auc: 0.7546
Epoch 5/40
176/176 [==============================] - 118s 668ms/step - loss: 0.6927 - auc: 0.5255 - val_loss: 0.6595 - val_auc: 0.7693
Epoch 6/40
176/176 [==============================] - 118s 670ms/step - loss: 0.6922 - auc: 0.5314 - val_loss: 0.6577 - val_auc: 0.7644
Epoch 7/40
176/176 [==============================] - 120s 681ms/step - loss: 0.6922 - auc: 0.5304 - val_loss: 0.6624 - val_auc: 0.7473
Epoch 8/40
176/176 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## SMOTE

In [32]:
X_train_2d.shape

(22410, 234, 236)

In [33]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(23786, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(23786, 36, 1)

In [35]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [36]:
y_train_2d.shape

TensorShape([23786, 2])

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [40]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [41]:
fp = f'./coughvid_hybrid_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2974/2974 [==============================] - 196s 61ms/step - loss: 0.7163 - auc: 0.4993 - val_loss: 0.6899 - val_auc: 0.5561
Epoch 2/40
2974/2974 [==============================] - 179s 60ms/step - loss: 0.6970 - auc: 0.5029 - val_loss: 0.6768 - val_auc: 0.6746
Epoch 3/40
2974/2974 [==============================] - 179s 60ms/step - loss: 0.6967 - auc: 0.5019 - val_loss: 0.6950 - val_auc: 0.5431
Epoch 4/40
2974/2974 [==============================] - 178s 60ms/step - loss: 0.6965 - auc: 0.4928 - val_loss: 0.7085 - val_auc: 0.3121
Epoch 5/40
2974/2974 [==============================] - 179s 60ms/step - loss: 0.6958 - auc: 0.5068 - val_loss: 0.6798 - val_auc: 0.6800
Epoch 6/40
2974/2974 [==============================] - 178s 60ms/step - loss: 0.6963 - auc: 0.4949 - val_loss: 0.6766 - val_auc: 0.7461
Epoch 7/40
2974/2974 [==============================] - 178s 60ms/step - loss: 0.6958 - auc: 0.5011 - val_loss: 0.6756 - val_auc: 0.6843
Epoch 8/40
2974/2974 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
1487/1487 [==============================] - 157s 101ms/step - loss: 0.7282 - auc: 0.4993 - val_loss: 0.6877 - val_auc: 0.6038
Epoch 2/40
1487/1487 [==============================] - 147s 99ms/step - loss: 0.6955 - auc: 0.5043 - val_loss: 0.7045 - val_auc: 0.3502
Epoch 3/40
1487/1487 [==============================] - 148s 100ms/step - loss: 0.6945 - auc: 0.5133 - val_loss: 0.6950 - val_auc: 0.4600
Epoch 4/40
1487/1487 [==============================] - 148s 99ms/step - loss: 0.6943 - auc: 0.5193 - val_loss: 0.7216 - val_auc: 0.3021
Epoch 5/40
1487/1487 [==============================] - 147s 99ms/step - loss: 0.6946 - auc: 0.5138 - val_loss: 0.7222 - val_auc: 0.3688
Epoch 6/40
166/166 [==============================] - 5s 28ms/step - loss: 0.7559 - auc: 0.3662


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
186/186 [==============================] - 136s 695ms/step - loss: 0.8444 - auc: 0.4785 - val_loss: 0.6762 - val_auc: 0.7260
Epoch 2/40
186/186 [==============================] - 127s 681ms/step - loss: 0.6954 - auc: 0.4991 - val_loss: 0.7080 - val_auc: 0.2528
Epoch 3/40
186/186 [==============================] - 126s 680ms/step - loss: 0.6938 - auc: 0.5182 - val_loss: 0.7255 - val_auc: 0.2619
Epoch 4/40
186/186 [==============================] - 126s 680ms/step - loss: 0.6911 - auc: 0.5416 - val_loss: 0.6815 - val_auc: 0.5582
Epoch 5/40
186/186 [==============================] - 126s 678ms/step - loss: 0.6867 - auc: 0.5641 - val_loss: 0.6597 - val_auc: 0.6373
Epoch 6/40
166/166 [==============================] - 4s 27ms/step - loss: 0.6607 - auc: 0.6348


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_39_hybrid_model_40_128_0.001/assets


## Evaluation

In [32]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [33]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [34]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [35]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 14s 28ms/step - loss: 0.6577 - auc: 0.7574


[0.6576951146125793, 0.757418155670166]

In [36]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6583 - auc: 0.7482


[0.6583083868026733, 0.7482160925865173]

In [37]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6754 - auc: 0.6352


[0.675421953201294, 0.6352189779281616]

In [38]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6596 - auc: 0.7514


[0.6596266031265259, 0.7514127492904663]

In [39]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6536 - auc: 0.7442


[0.6535892486572266, 0.7441530227661133]

### Raw features

In [40]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [41]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6384 - auc: 0.7409


[0.6383568048477173, 0.7409135103225708]

In [42]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5821 - auc: 0.7656


[0.5820732712745667, 0.765593409538269]

In [43]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5798 - auc: 0.7765


[0.5798322558403015, 0.7765454649925232]

In [44]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5927 - auc: 0.7608


[0.5927251577377319, 0.7607962489128113]

In [45]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5825 - auc: 0.7607


[0.5824873447418213, 0.7607284188270569]

### SMOTE

In [46]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6607 - auc: 0.6348


[0.6607256531715393, 0.6348146796226501]

In [47]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.7533 - auc: 0.4268


[0.7532990574836731, 0.42679062485694885]

In [48]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6923 - auc: 0.5670


[0.6923022866249084, 0.5669546127319336]

In [49]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.7559 - auc: 0.3662


[0.7559055089950562, 0.36617782711982727]

In [50]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6971 - auc: 0.3719


[0.6970987319946289, 0.37189269065856934]